In [2]:
import pandas as pd

#set up the connection to google drive and import the dataset
from google.colab import drive
drive.mount('/content/drive')

filedir = '/content/drive/MyDrive/fwe458/datasets/'
fname = filedir + "Wcr_GPPdaily.csv"

df = pd.read_csv(fname)

#display the first few rows
df.head()

Mounted at /content/drive


,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240


In [3]:
#rename the columns
df = df.rename(columns = {'TA_F': 'TA', 'SW_IN_F': 'SW', 'VPD_F': 'VPD', 'GPP_NT_VUT_REF': 'GPP'})

#remove the timestamp column
df.drop(columns=['TIMESTAMP'], inplace=True)

#display the first few rows of the updated data frame
df.head()

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


In [4]:
#remove any rows with missing values
df.dropna(inplace=True)

In [5]:
#add new columns that are made up of interaction terms
df["SW_x_VPD"] = df["SW"] * df["VPD"]
df["SW_x_TA"] = df["SW"] * df["TA"]
df["VPD_x_TA"] = df["VPD"] * df["TA"]

#display the first few rows of the updated data frame
df.head()

,TA,SW,VPD,GPP,SW_x_VPD,SW_x_TA,VPD_x_TA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209


In [10]:
import numpy as np
from scipy.optimize import minimize

#assign each of the columns needed for linear regression to a variable
SW = df["SW"]
VPD = df["VPD"]
TA = df["TA"]
SW_x_VPD = df["SW_x_VPD"]
SW_x_TA = df["SW_x_TA"]
VPD_x_TA = df["VPD_x_TA"]
GPP = df["GPP"]

#create a function for the linear model
def lin_model(params, SW, VPD, TA, SW_x_VPD, SW_x_TA, VPD_x_TA, GPP):

  #set coefficients equal to params
  b0, b1, b2, b3, b4, b5, b6 = params
  #define the linear model equation
  GPPpred = b0 + b1*SW + b2*VPD + b3*TA + b4*SW_x_VPD + b5*SW_x_TA + b6*VPD_x_TA

  #return the mean squared error
  return (np.mean((GPPpred - GPP) **2))**0.5

#initialize params to all be zero
initial_params = np.zeros(7)

#minimze the function and assign the results to a variable
result = minimize(lin_model, initial_params, args=(SW, VPD, TA, SW_x_VPD, SW_x_TA, VPD_x_TA, GPP) )

#assign variables to the resulting coefficients
b0, b1, b2, b3, b4, b5, b6 = result.x
#print optimized coefficients
print(result.x)

[-9.44238554e+02 -3.24857686e+00 -6.48473871e+02  8.21606704e+01
  1.79217770e+00 -4.56001422e-01  1.35942273e+01]


In [11]:
from sklearn.linear_model import LinearRegression

#define independent and dependent variables
X = df[['SW', 'VPD', 'TA', 'SW_x_VPD','SW_x_TA','VPD_x_TA' ]]
y = df['GPP']

#fit the model
model = LinearRegression()
model.fit(X, y)

#print the model parameters
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Coefficients: [-3.24914968e+00 -6.48574173e+02  8.21699813e+01  1.79249151e+00
 -4.56050357e-01  1.35954025e+01]
Intercept: -944.0672018203164


In [12]:
#compute the r-squared value for sklearn's LinearRegression function
r2 = model.score(X,y)
r2

0.013582700933254976

In [13]:
#computing the r-squared value for SciPy's optimization method

#find the predicted GPP using the coefficients obtained by the model
GPPpred = b0 + b1*SW + b2*VPD + b3*TA + b4*SW_x_VPD + b5*SW_x_TA + b6*VPD_x_TA

#define a function to find the r_squared value
def r_squared(GPP, GPPpred):
    #find the residual sum of squares
    rss = np.sum((GPP - GPPpred)**2)
    #find the total sum of squares
    tss = np.sum((GPP - np.mean(GPP))**2)
    #calculate and return the r-squared value
    return 1 - (rss / tss)

#call the r_squared function
r2 = r_squared(GPP, GPPpred)
print(r2)


0.013582700643345214


The coefficients obtained by SciPy's optimization method and sklearn's LinearRegression function are nearly identical. They are the same up to 2 or 3 decimal points depending on the coefficient.

The R-squared value for both models is even more similar, identical up to 9 decimal points.